### Convert Welcome Week Schedule (.docx) into a calender file importable by google calendar (and others)

In [136]:
import re
from bs4 import BeautifulSoup
import requests

html_path = "/tmp/timetable.html"
ics_path = "/tmp/timetable.ics"

with open(html_path, 'r') as file:
    data = file.read()

# Parse the html content
soup = BeautifulSoup(data, "lxml")

clean_time_re = re.compile("[^0-9]")
def split_time(t):
    t = clean_time_re.sub("", t)
    t = t.replace("00", ":00 ")
    t = t.replace("30", ":30 ")
    times = []
    for time in t.strip().split(" "):
        t1 = time.split(":")
        if int(t1[0]) < 7:
            times.append("%d:%s" % (int(t1[0])+12, t1[1]))
        else:
            times.append(time)
    return times

def clean_text(t):
    t = t.replace("\n", " ")
    t = t.replace("  ", " ")
    return t


events = []
for rowIndex, row in enumerate(soup.find("table").find_all("tr")):
    r = []
    for colIndex, td in enumerate(row.find_all("td")):
        text = clean_text(td.text).strip()
        href = ""
        for link in td.find_all("a"):
            href = link.get("href")
        r.append(text)
        
        if rowIndex and colIndex and len(text):
            title = text.split("\n")[0]
            times = split_time(r[0])
            event = {"date":heading[colIndex], "from": times[0].strip(), "to": times[1].strip(), "title": title, "text": text, "href": href}
            events.append(event)
        
    if not rowIndex:
        heading = r
print("read %d events" % len(events))
print(events[0])


read 15 events
{'date': '14/09/20', 'from': '11:00', 'to': '12:00', 'title': 'Meet your course leader and fellow  students All your questions answered by your course leader  – and start to get to know your fellow students. Link to the  meeting on Teams', 'text': 'Meet your course leader and fellow  students All your questions answered by your course leader  – and start to get to know your fellow students. Link to the  meeting on Teams', 'href': 'https://teams.microsoft.com/l/meetup-join/19%3ameeting_NzlmMTc0NjYtNjM3Zi00OTgzLWJlMTctNDVkMzEzZDU2NWRm%40thread.v2/0?context=%7b%22Tid%22%3a%227e2be055-828a-4523-b5e5-b77ad9939785%22%2c%22Oid%22%3a%22105033a2-5ca7-410d-9739-440f0056562b%22%7d'}


In [146]:
from ics import Calendar, Event
from datetime import datetime
from dateutil import tz
date_format = '%Y-%m-%d %H:%M:%S' # Format the ics module expects the dates to be in

def make_time(event, key):
    date_toks = event["date"].split("/")
    t = "20%s-%s-%s %s:00" % (date_toks[2], date_toks[1], date_toks[0], event[key])
    dt = datetime.strptime(t,date_format).replace(tzinfo=tz.gettz('BST'))
    dt = dt.astimezone(tz.tzutc())
    return dt.strftime(date_format)

c = Calendar()
for event in events:
    e = Event()
    e.name = event["text"]
    e.begin = make_time(event, "from")
    e.end = make_time(event, "to")
    e.url = event["href"]
    c.events.add(e)

with open(ics_path, 'w') as my_file:
    my_file.writelines(c)
    
with open(ics_path) as myfile:
    [print(next(myfile).strip()) for x in range(8)]


BEGIN:VCALENDAR
VERSION:2.0
PRODID:ics.py - http://git.io/lLljaA
BEGIN:VEVENT
DTEND:20200914T130000Z
DTSTART:20200914T120000Z
SUMMARY:Meet  your Student Success Advisers (SSAs) SSAs are Arts\, Design and Media graduates who can  provide lots of insider tips and support.   Link to the Activity on Teams
UID:0dd7e0cb-5f27-4a23-87d2-ad70903d5380@0dd7.org
